In [1]:
pip install requests beautifulsoup4 selenium boto3


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 12.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 KB 9.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 24.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 5.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ['PATH'] += os.pathsep + '/home/workbench/.local/bin'


In [3]:
import os
print(os.environ['PATH'])


/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/workbench/.local/bin


In [11]:
import requests
from bs4 import BeautifulSoup

# URL of the job postings page
url = "https://www.indeed.com/jobs?q=software+engineer&l=chicago"  # Replace with the actual URL you want to scrape

# Send a GET request to fetch the page content
response = requests.get(url)

# Parse the page content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find job postings (Assuming job listings are inside a class named 'job-listing')
jobs = soup.find_all('div', class_='job-listing')

# Extract job details
for job in jobs:
    title = job.find('h2').text  # Assuming job title is inside <h2> tags
    company = job.find('h3').text  # Assuming company name is inside <h3> tags
    description = job.find('p').text  # Assuming description is inside <p> tags
    print(f"Job Title: {title}, Company: {company}, Description: {description}")


In [17]:
import csv
import requests
from bs4 import BeautifulSoup

# List of URLs to scrape
urls = [
    'https://www.careerjet.com/search/jobs?s=software+engineer&l=chicago',
    'https://stackoverflow.com/jobs?q=python&l=chicago',
    'https://www.indeed.com/jobs?q=software+engineer&l=chicago'# Replace with actual URL
    # Add more URLs as needed
]

# Open a CSV file and write the headers
with open('jobs.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Job Title', 'Company', 'Location'])  # CSV headers

    # Loop through each URL
    for url in urls:
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find all job postings (adjust the selector as needed)
            jobs = soup.find_all('article', class_='job')

            # Loop through each job and extract details
            for job in jobs:
                try:
                    # Check if the title, company, and location exist before accessing them
                    title = job.find('h2').text.strip() if job.find('h2') else 'N/A'
                    company = job.find('p', class_='company').text.strip() if job.find('p', class_='company') else 'N/A'
                    location = job.find('p', class_='location').text.strip() if job.find('p', class_='location') else 'N/A'

                    # Write the job details to the CSV file
                    writer.writerow([title, company, location])
                except AttributeError:
                    # Handle cases where an element might be missing
                    print(f"Error scraping one of the job entries from URL: {url}")
                    continue

            print(f"Data successfully scraped from {url}")

        except Exception as e:
            # Handle errors related to URL fetching or parsing
            print(f"Error fetching or parsing URL: {url}")
            print(e)

print("Data written to jobs.csv successfully.")


Data successfully scraped from https://www.careerjet.com/search/jobs?s=software+engineer&l=chicago
Data successfully scraped from https://stackoverflow.com/jobs?q=python&l=chicago
Data successfully scraped from https://www.indeed.com/jobs?q=software+engineer&l=chicago
Data written to jobs.csv successfully.


In [20]:
import csv
import requests
from bs4 import BeautifulSoup

# List of URLs to scrape
urls = [
    'https://www.careerjet.com/search/jobs?s=software+engineer&l=chicago',
    'https://www.indeed.com/q-Software-Engineer-l-Chicago,-IL-jobs.html',  # Replace with actual URL
    'https://stackoverflow.com/jobs?sort=p'  # Replace with actual URL
    # Add more URLs as needed
]

# Function to parse job postings from a specific URL
def parse_jobs(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Adjust the selectors based on the site’s structure
        jobs = []
        if 'careerjet' in url:
            # Example for Careerjet
            job_elements = soup.find_all('article', class_='job')
            for job in job_elements:
                title = job.find('h2').text.strip() if job.find('h2') else 'N/A'
                company = job.find('p', class_='company').text.strip() if job.find('p', class_='company') else 'N/A'
                location = job.find('p', class_='location').text.strip() if job.find('p', class_='location') else 'N/A'
                jobs.append([title, company, location])
        
        elif 'indeed' in url:
            # Example for Indeed
            job_elements = soup.find_all('div', class_='jobsearch-SerpJobCard')
            for job in job_elements:
                title = job.find('a', class_='jobtitle').text.strip() if job.find('a', class_='jobtitle') else 'N/A'
                company = job.find('span', class_='company').text.strip() if job.find('span', class_='company') else 'N/A'
                location = job.find('div', class_='location').text.strip() if job.find('div', class_='location') else 'N/A'
                jobs.append([title, company, location])
        
        elif 'stackoverflow' in url:
            # Example for Stack Overflow
            job_elements = soup.find_all('div', class_='js-result')
            for job in job_elements:
                title = job.find('a', class_='s-link').text.strip() if job.find('a', class_='s-link') else 'N/A'
                company = job.find('div', class_='fc-black-700').text.strip() if job.find('div', class_='fc-black-700') else 'N/A'
                location = job.find('span', class_='fc-black-500').text.strip() if job.find('span', class_='fc-black-500') else 'N/A'
                jobs.append([title, company, location])
        
        return jobs
    except Exception as e:
        print(f"Error fetching or parsing URL: {url}")
        print(e)
        return []

# Open a CSV file and write the headers
with open('jobs.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Job Title', 'Company', 'Location'])  # CSV headers

    # Loop through each URL
    for url in urls:
        jobs = parse_jobs(url)
        for job in jobs:
            writer.writerow(job)
        print(f"Data successfully scraped from {url}")

print("Data written to jobs.csv successfully.")


Data successfully scraped from https://www.careerjet.com/search/jobs?s=software+engineer&l=chicago
Data successfully scraped from https://www.indeed.com/q-Software-Engineer-l-Chicago,-IL-jobs.html
Data successfully scraped from https://stackoverflow.com/jobs?sort=p
Data written to jobs.csv successfully.
